Connect to drive, set directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/Thesis_UU/push/3group'

Import libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#### Sampling - so that papers which are in train set are not in test set

Splitting data into train and test set

In [ ]:
sample_papers= pd.read_csv('Data (CSV)/papers_journal_balanced.csv')

In [ ]:
sample_train = sample_papers.sample(frac=0.75, random_state=1)

In [ ]:
len(sample_train)

In [ ]:
X_train = list(sample_train.preprocessed.values)
y_train = list(sample_train['class'].values)

In [ ]:
sample_test = sample_papers[~sample_papers.ID.isin(sample_train.ID)]

In [ ]:
len(sample_test)

In [ ]:
X_test = list(sample_test.preprocessed.values)
y_test = list(sample_test['class'].values)

tf-idf - vectorizing the text

In [ ]:
tfidf = TfidfVectorizer() 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

Hyperparameter tuning

Code for hyperparameter tuning based on https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=kf, scoring='f1_weighted',error_score=0)
grid_result = grid_search.fit(X_train_tfidf, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fit LR classifier

In [ ]:
lr = LogisticRegression(solver='newton-cg', penalty="l2", C=100)

lr.fit(X_train_tfidf, y_train)

Predict classes for test set

In [ ]:
labels = ['error', 'misconduct', 'non_retracted']

In [ ]:
y_pred = lr.predict(X_test_tfidf)

print(classification_report(y_test, y_pred, 
                          target_names=labels))

Confusion matrix

In [ ]:
print(confusion_matrix(y_test, y_pred))

Top 30 indicative words

In [ ]:
feature_array = np.array(tfidf.get_feature_names())
tfidf_sorting = np.argsort(X_test_tfidf.toarray()).flatten()[::-1]

n = 30
top_n = feature_array[tfidf_sorting][:n]

In [ ]:
top_n

## External validation

In [ ]:
test_dataset = pd.read_csv('Data (CSV)/journal_test_data_set.csv', encoding="utf-8-sig")

In [ ]:
test_dataset.head(2)

In [ ]:
len(test_dataset)

Redefining test data

In [ ]:
X_test= list(test_dataset.preprocessed.values)
y_test = list(test_dataset['class'].values)

tf-idf - vectorizing the text

In [ ]:
X_test_tfidf = tfidf.transform(X_test)

Predict classes for external validation dataset

In [ ]:
y_pred = lr.predict(X_test_tfidf)

print(classification_report(y_test, y_pred, 
                          target_names=labels))

Confusion matrix

In [ ]:
print(confusion_matrix(y_test, y_pred))